## Demo

**Dataset**: Telco Customer Churn

Cada fila representa un cliente, cada columna contiene los atributos del cliente.

El conjunto de datos incluye información sobre:

- Clientes que se fueron en el último mes: Churn

- Servicios a los que cada cliente se ha suscrito: teléfono, líneas múltiples, Internet, seguridad en línea, respaldo en línea, protección de dispositivos, soporte técnico y transmisión de TV y películas.

- Información de la cuenta del cliente: cuánto tiempo ha sido cliente, contrato, método de pago, facturación electrónica, cargos mensuales y cargos totales

- Información demográfica sobre los clientes: género, rango de edad y si tienen parejas y dependiente

Infomacion dataset en el siguiente enlace: [link](https://www.kaggle.com/datasets/blastchar/telco-customer-churn)

In [1]:
!pip install -q awswrangler

In [2]:
import awswrangler as wr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams["figure.figsize"] =8,4 # para ajustar el tamaño de imagen
import warnings
warnings.filterwarnings("ignore") #para evitar errores de operaciones matematicas
import random
import statistics
import math
#from pandas_profiling import ProfileReport

In [3]:
dataset = wr.s3.read_csv(path='s3://data-tecnicas-muestreo/datasets/Telco_customer_churn.csv')

In [4]:
display(dataset.head())
display(dataset.shape)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,Premium Tech Support,Unlimited Data,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Satisfaction Score,Customer Status,Churn Score.1,Churn Category
0,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,Yes,0.0,0,20.94,129.09,1,Churned,86,Competitor
1,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,Yes,0.0,0,18.24,169.89,2,Churned,67,Other
2,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,...,No,Yes,0.0,0,97.20,917.70,3,Churned,86,Other
3,7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,...,Yes,Yes,0.0,0,136.92,3182.97,3,Churned,84,Other
4,0280-XJGEX,Male,0,No,No,49,Yes,Yes,Fiber optic,No,...,No,Yes,0.0,0,2172.17,7208.47,1,Churned,89,Competitor


(7043, 52)

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 52 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   customerID                         7043 non-null   object 
 1   gender                             7043 non-null   object 
 2   SeniorCitizen                      7043 non-null   int64  
 3   Partner                            7043 non-null   object 
 4   Dependents                         7043 non-null   object 
 5   tenure                             7043 non-null   int64  
 6   PhoneService                       7043 non-null   object 
 7   MultipleLines                      7043 non-null   object 
 8   InternetService                    7043 non-null   object 
 9   OnlineSecurity                     7043 non-null   object 
 10  OnlineBackup                       7043 non-null   object 
 11  DeviceProtection                   7043 non-null   objec

### Ubicamos la variable de interes para la estimacion puntual (media aritmetica)

In [6]:
dataset["tenure"].describe()

count    7043.000000
mean       32.371149
std        24.559481
min         0.000000
25%         9.000000
50%        29.000000
75%        55.000000
max        72.000000
Name: tenure, dtype: float64

### Ubicamos la variable de interes para la estimacion proporcional (variable dicotomica)

In [7]:
dataset['churn_rate'].value_counts()

0    5174
1    1869
Name: churn_rate, dtype: int64

### Estimacion puntual 

#### Tenure: Número de meses que el cliente ha permanecido en la empresa

In [24]:
target = dataset['tenure']

In [25]:
        
def sistematico_puntual(target, LEE):
    
    mean_poblacional = target.mean()    # promedio poblacional (variable de interes)
    N = len(target)   # tamaño poblacional
    D =  pow(LEE, 2)/4    # limite margenes error estimacion poblacional
    var = statistics.variance(target)   # Varianza Poblacional
    
    n =  math.floor((N*var)/((N-1)*D + var)) # seleccion de muestra aleatoria
     
    rango = math.floor(N/n)  # rango para seleccionar una valor aleatorio de cero a "K"
       
    random.seed(123)   
    
    k = int(''.join(map(str, random.sample (range(1, rango + 1), 1)) ) ) # se selecciona un valor aleatorio k <= N/n      
    

    lista_aleatoria = list(range(k, N, rango))  # se define la secuencia de k hasta N de rango en rango
       
    target_sample = target[lista_aleatoria]  # # seleccion muestra por index
    

    mean_est = target_sample.mean()  # promedio muestral estimado (variable de interes)
    
    var_mean_est = (np.var(target_sample)/n)*(1-(n/N))  # varianza muestral estimada (variable de interes)
    error_est =  2*math.sqrt(var_mean_est) # error de estimacion muestra estimada (variable de interes)

    LI =  mean_est - error_est  # limite superior estimado estimacion poblacional
    LS =  mean_est + error_est  # limite inferior estimado estimacion poblacional
    
    data_filtrada = dataset.iloc[target_sample.index, :]  
    
    print("El intervalo de confianza de la proporcion esta entre", "[",LI, LS,"]")
    print("tamaño total poblacional:" , len(dataset) )
    print("tamaño de muestra estimada:" , len(data_filtrada) )
    print("promedio poblacional estimado:" , mean_poblacional )
    print("promedio muestral estimado:" , mean_est )
    print("error de estimacion:" , error_est )
    
    return(data_filtrada)
    
    


In [26]:
df_sistematico_puntual = sistematico_puntual(target, 1.1)

El intervalo de confianza de la proporcion esta entre [ 31.215262185017192 33.39121140953136 ]
tamaño total poblacional: 7043
tamaño de muestra estimada: 1761
promedio poblacional estimado: 32.37114865824223
promedio muestral estimado: 32.30323679727427
error de estimacion: 1.087974612257083


In [27]:
df_sistematico_puntual.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,Premium Tech Support,Unlimited Data,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Satisfaction Score,Customer Status,Churn Score.1,Churn Category
1,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,Yes,0.00,0,18.24,169.89,2,Churned,67,Other
5,4190-MFLUW,Female,0,Yes,Yes,10,Yes,No,DSL,No,...,Yes,Yes,0.00,0,94.10,622.45,1,Churned,78,Competitor
9,8665-UTDHZ,Male,0,Yes,Yes,1,No,No phone service,DSL,No,...,No,Yes,0.00,0,0.00,30.20,1,Churned,90,Competitor
13,8168-UQWWF,Female,0,No,No,11,Yes,Yes,Fiber optic,No,...,No,Yes,0.00,0,303.27,1408.67,3,Churned,70,Competitor
17,1658-BYGOY,Male,1,No,No,18,Yes,Yes,Fiber optic,No,...,No,Yes,45.61,0,203.94,1910.88,2,Churned,81,Competitor


### Interpretacion:



En resumen, la muestra sistematica nos indica que la cantidad de tiempo promedio que los clientes han permanecido en la empresa es de 32 (32.30) meses, osea alrededore de 2 años y ocho meses. Haciendo un comparativo la estimacion muestral se asemeja mucho a la metrica poblacional que fue de 32.37 meses.

Es necesario aproximadamente 1761 observaciones (clientes distintos) para estimar la cantidad promedio de meses que el cliente ha permanecido en la empresa, teniedo en cuenta un error de 1.087.

Entre mas pequeño sea el LEE (limite de error de estimacion), el tamaño de muestra aumentara.

Ademas, la estimacion promedio muestral sistematica se encuentra dentro de intervalo de confianza estimado,entonces podemos asegurar que 95 de cada 100 observaciones (clientes) contiene la cantidad promedio verdadera de meses que el cliente ha permanecido en la empresa (poblacional) 


### Guardamos el dataset muestra en la ruta S3

In [28]:
wr.s3.to_csv(df = df_sistematico_puntual, path = 's3://data-tecnicas-muestreo/datasets/Telco_customer_churn_sistematico_puntual.csv', index=False)

{'paths': ['s3://data-tecnicas-muestreo/datasets/Telco_customer_churn_sistematico_puntual.csv'],
 'partitions_values': {}}

### Estimacion proporcional

#### Si el cliente abandona o no (Sí = 1 o No = 0)

In [31]:
target = dataset['churn_rate']

In [32]:

def sistematico_proporcional(target, LEE):
    
     p = target.mean()   # proporcion casos positivos (valores 1) poblacional
     q = 1 - p    # complemento proporcion CP poblacional
     N = len(target)   # tamaño poblacional
     D =  pow(LEE, 2)/4    # limite margenes error estimacion poblacional
     var = statistics.variance(target)   # Varianza Poblacional
     
     n =  math.floor((N*var)/((N-1)*D + var)) # seleccion de muestra aleatoria
      
     rango = math.floor(N/n)  # rango para seleccionar una valor aleatorio k
     
     k = int(''.join(map(str, random.sample (range(1, rango + 1), 1)) ) ) # se selecciona un valor aleatorio k <= N/n      
     

     lista_aleatoria = list(range(k, N, rango))  # se define la secuencia de k hasta N de rango en rango
        
     target_sample = target[lista_aleatoria]  # # seleccion muestra por index
    

     p_est = target_sample.mean()  # proporcion casos positivos estimados muestra
     q_est =  1 - p_est   # complemento proporcion casos positivos estimados muestra
     var_p_est = ((p_est*q_est)/(n-1))/(1 - n/N) # varianza proporcion CP estimada muestra
     error_est =  2*math.sqrt(var_p_est)  # error de estimacion proporcion CP

     LI =  p_est - error_est  # limite superior estimado estimacion poblacional
     LS =  p_est + error_est  # limite inferior estimado estimacion poblacional
     
     data_filtrada = dataset.iloc[target_sample.index, :]  
     
     print("El intervalo de confianza de la proporcion esta entre", "[",LI, LS,"]")
     print("tamaño total poblacional:" , len(dataset) )
     print("tamaño de muestra estimada:" , len(data_filtrada) )
     print("proporcion casos positivos poblacional:" , p )
     print("proporcion casos positivos estimados:" , p_est )
     print("error de estimacion:" , error_est )
     
     return(data_filtrada)

In [36]:
df_sistematico_proporcional = sistematico_proporcional(target, 0.020)

El intervalo de confianza de la proporcion esta entre [ 0.23979834573928172 0.29088347244253643 ]
tamaño total poblacional: 7043
tamaño de muestra estimada: 1760
proporcion casos positivos poblacional: 0.2653698707936959
proporcion casos positivos estimados: 0.26534090909090907
error de estimacion: 0.02554256335162737


In [37]:
df_sistematico_proporcional.churn_rate.value_counts()

0    1293
1     467
Name: churn_rate, dtype: int64

In [38]:
dataset.churn_rate.value_counts()

0    5174
1    1869
Name: churn_rate, dtype: int64

In [39]:
display(df_sistematico_proporcional.head())
print(df_sistematico_proporcional.shape)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,Premium Tech Support,Unlimited Data,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Satisfaction Score,Customer Status,Churn Score.1,Churn Category
3,7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,...,Yes,Yes,0.0,0,136.92,3182.97,3,Churned,84,Other
7,1066-JKSGK,Male,0,No,No,1,Yes,No,No,No internet service,...,No,No,0.0,0,33.58,53.73,1,Churned,92,Competitor
11,6047-YHPVI,Male,0,No,No,5,Yes,No,Fiber optic,No,...,No,Yes,0.0,0,63.00,379.90,1,Churned,66,Competitor
15,9420-LOJKX,Female,0,No,No,15,Yes,No,Fiber optic,Yes,...,No,Yes,0.0,0,59.40,1485.80,1,Churned,82,Competitor
19,5919-TMRGD,Female,0,No,Yes,1,Yes,No,Fiber optic,No,...,No,Yes,0.0,0,13.13,92.48,1,Churned,87,Competitor


(1760, 52)


### Interpretacion:



La muestra proporcional sistematica nos indica que 26.53% de observaciones (clientes) decidio abandonar la empresa de telecomunicaciones, esta propocional se asemeja mucho probabilidad real de abandono poblacional de 26.53%.

Es necesario aproximadamente 1760 observaciones (clientes distintos) para estimar la proporcional de clientes que abandonaron la empresa en el ultimo año, teniedo en cuenta un error de 0.025.

Entre mas pequeño sea el LEE (limite de error de estimacion), el tamaño de muestra aumentara.

Ademas, la estimacion proporcional muestral sistematica de clientes que abandonaron la empresa se encuentra dentro de intervalo de confianza estimado, entonces podemos asegurar que 95 de cada 100 observaciones de clientes que abandonaron la empresa en la muestra son verdaderamente clientes que abandonaron la empresa en la poblacion. 


In [40]:
wr.s3.to_csv(df = df_mas_proporcional, path = 's3://data-tecnicas-muestreo/datasets/Telco_customer_churn_sistematico_proporcional.csv', index=False)

{'paths': ['s3://data-tecnicas-muestreo/datasets/Telco_customer_churn_sistematico_proporcional.csv'],
 'partitions_values': {}}